In [55]:
import os

from sqlalchemy import create_engine, text
import pandas as pd
from dotenv import load_dotenv

load_dotenv("../.env")


In [ ]:
METRICS_USER = os.environ["METRICS_USER"]
METRICS_PASSWORD = os.environ["METRICS_PASSWORD"]
METRICS_HOSTNAME = os.environ["METRICS_HOSTNAME"]
METRICS_PORT = os.environ["METRICS_PORT"]
METRICS_DB = os.environ["METRICS_DB"]

In [ ]:
engine = create_engine(
    f"postgresql://{METRICS_USER}:{METRICS_PASSWORD}@{METRICS_HOSTNAME}:{METRICS_PORT}/{METRICS_DB}"
)

# TMP

In [56]:

def default_metrics_extraction(query: str ):
    with engine.connect() as connection:
        df_raw = pd.read_sql_query(text(query), connection)
    return df_raw

In [8]:
bdp = default_metrics_extraction("""
                SELECT
      COUNT(DISTINCT p1.id) AS unique_patient_count,
      fhirql_code(ccc0.code) AS "ccc0_code"
    FROM
      condition c0
    JOIN condition_code cc0 ON cc0._resource = c0._id
    JOIN condition_code_coding ccc0 ON ccc0._resource = c0._id
    JOIN condition_subject cs0 ON cs0._resource = c0._id
    JOIN patient p1 ON p1._id = cs0._reference_id
      AND cs0._reference_type = 'Patient'
    WHERE
      fhirql_code(ccc0.code) IN ('C92.40')
    GROUP BY
      fhirql_code(ccc0.code);
  """
)